In [1]:
from itertools import product
import numpy as np
import scipy as sp
from scipy import optimize
from copy import deepcopy
import json
from datetime import datetime
from sklearn import preprocessing
import time
from cvxopt import matrix, solvers
import math
from scipy import linalg
import sys
import pandas as pd

In [2]:
def irl(mdp,trajectories,threshold,learning_rate,model,log='false',initial_input=[],return_num_iter=False):
    '''
    Find the reward parameter theta using given MDP and trajectories.
    
    mdp:The Markov Decision Process which needed to be solved
    
    trajectories: observed trajectories, sequence of state and action (s1,a1,s2,a3,...,sn)
    '''
    
    feature_matrix=mdp.feature_matrix
    discount=mdp.discount
    
    #get number of features
    n_feature=len(feature_matrix[0])
    
    #Randomnize initial reward parameter
    if len(initial_input)==0:
        theta=np.random.random(size=(n_feature,))
    else:
        theta=initial_input
    
    
    #Get the feature expectation, which is \tilde(phi)
    feature_expectation=find_feature_expectation(mdp, trajectories)
    #theta=feature_expectation
    
    start,end=get_distribution(mdp,trajectories)
    #Gredient decent model
    
    sign=True
    it_count=0
    last_diff=sys.maxsize
    vi_de=[0]*len(mdp.feature_matrix)
    for trajectory in trajectories:
        for state_action in trajectory:
            vi_de[mdp.state_action_index[state_action[0]][state_action[1]]]+=1
    for i in range(len(vi_de)):
        vi_de[i]/=len(trajectories)
    po_de={}
    for state_i in range(len(mdp.states)):
        po_ac={}
        sum_vi=0
        for action in mdp.state_action[str(state_i)]:
            sum_vi+=vi_de[mdp.state_action_index[state_i][action]]
        for action in mdp.state_action[str(state_i)]:
            po_ac[action]=vi_de[mdp.state_action_index[state_i][action]]/sum_vi
        po_de[state_i]=po_ac
    reward_log=[]
    error_log=[]
    policy_log=[]
    while sign:
        iterated_reward=np.dot(feature_matrix,theta)
        
        #solve visiting frequency
        #A,state_visiting_frequency
        if model=='paper':
            po,result=recover_policy(mdp,iterated_reward,start,end)
        elif model=='our':
            po,result=recover_policy_value(mdp,iterated_reward,start,end)
        state_visiting_frequency=result
        #sum_visit=sum(state_visiting_frequency)
        state_visiting_probability=state_visiting_frequency
        #state_visiting_probability=[x/sum_visit for x in state_visiting_frequency]
        #state_visiting_probability=[x/sum(state_visiting_frequency) for x in state_visiting_frequency]
        
        policy_error=0
        for p in po:
            for k in po[p]:
                policy_error+=abs(po[p][k]-po_de[p][k])
        
        gradient=feature_expectation-np.dot(np.array(feature_matrix).transpose(),state_visiting_probability)
        #gradient=feature_expectation-feature_matrix.T.dot(state_visiting_frequency)       
        old_theta=deepcopy(theta)
        theta+=learning_rate*gradient
        gra_sum=sum(gradient)
        abs_sum=0
        for it in theta:
            abs_sum+=abs(it)
        total_reward=np.dot(iterated_reward,state_visiting_probability)
        theta_diff=judge_threshold(gradient,[0]*len(gradient))
        policy_log.append(policy_error/len(state_visiting_probability))
        reward_log.append(total_reward)
        error_log.append(theta_diff)
        
        if abs(theta_diff-last_diff)<threshold:
            sign=False
        if theta_diff>last_diff:
            learning_rate/=2
        last_diff=theta_diff
        it_count+=1
        if it_count%200==0:
            print(it_count)
            print(last_diff)
            print(policy_error/len(state_visiting_probability))
    if return_num_iter:
        print(it_count)
        return theta,last_diff,it_count
    if log=='false':
        return theta,last_diff
    elif log=='true':
        return theta,last_diff,reward_log,error_log,policy_log
            

In [3]:
def find_feature_expectation(mdp, trajectories):
    '''
    Find the feature expectation for the given trajectories. Just a average over all feature vector of (s,a) pairs in trajectories.
    
    INPUT:
    mdp: the Markov Decision Process which needed to be solved
    
    state_action_index: the index of feature, phi(s,a) in the feature matrix
    
    OUTPUT:
    feature_expectation: vector average feature vector of all trajectories
    
    '''
    feature_matrix=mdp.feature_matrix
    state_action_index=mdp.state_action_index
    
    feature_expectation=np.zeros(len(feature_matrix[0]))
    
    for trajectory in trajectories:
        discount=1
        for state_action in trajectory:
            feature_vector=[x*discount for x in feature_matrix[state_action_index[state_action[0]][state_action[1]]]]
            feature_expectation=feature_vector+feature_expectation
            discount*=mdp.discount
    
    feature_expectation/= len(trajectories)
    
    return feature_expectation

In [4]:
def svf_sovler(mdp,policy,start_distribution,discount_type='no'):
    '''
    Find visiting frequency of each (s,a) pair
    
    INPUT:
    mdp: Markov Decision Process
       
    OUTPUT:
    frequency vector: visting frequency of each (s,a) pair
    '''
    
    n_states=len(mdp.states)
    state_action=mdp.state_action
    transition_matrix=mdp.transition_matrix
    if discount_type=='no':
        discount=1
    else:
        discount=mdp.discount
    start_distribution=deepcopy(start_distribution)
    n_start=len(start_distribution)
    for i in range(n_states-n_start):
        start_distribution.append(0)
        
    #Note linear programming constraint A*x=b
    #Firstly construct coeffient matrix A
    A=[]
    idx=[]
    for next_state_i in range(n_states):
        a=[]
        sum_s=0
        for state_i in range(n_states):
            if next_state_i!=state_i:
                for action in state_action[str(state_i)]:
                    a.append(0-discount*transition_matrix[state_i][action][next_state_i])
            else:
                for action in state_action[str(state_i)]:
                    a.append(1-discount*transition_matrix[state_i][action][next_state_i])
                    sum_s+=1-discount*transition_matrix[state_i][action][next_state_i]
        A.append(a)
    b=deepcopy(start_distribution)
    out=True
    for state_i in range(n_states):
        skip=False
        for action in state_action[str(state_i)]:
            a=[0]*len(A[0])
            if not skip and mdp.transition_matrix[state_i][action][state_i]<1:
                skip=True
                continue
            for action_p in state_action[str(state_i)]:
                if action==action_p:
                    a[mdp.state_action_index[state_i][action_p]]=policy[state_i][action]-1
                else:
                    a[mdp.state_action_index[state_i][action_p]]=policy[state_i][action]
            b.append(0)
            A.append(a)
    #Then, reverse the reward function since scipy minimize the object function
    try:
        re=linalg.solve(A,b)
    except:
        '''
        for i in range(len(A[0])):
            for j in range(len(A[0])):
                if i!=j:
                    sum_1=0
                    for t in range(len(A)):
                        if A[t][i]!=A[t][j]:
                            sum_1+=1
                    if sum_1==0:
                        print(i,j)
        '''           
        s=[A[x][9] for x in range(len(A)) ]
        print(s)
        
        raise(ValueError('no'))
    return re

In [5]:
def judge_threshold(original,new):
    result=0
    for i in range(len(original)):
        result+=abs(original[i]-new[i])**2
    return result

In [6]:
def judge_1norm_threshold(original,new):
    result=0
    for i in range(len(original)):
        result+=abs(original[i]-new[i])
    return result

In [7]:
def judge_max_norm(first,second):
    if len(first)!=len(second):
        raise(ValueError('Not same length'))
    else:
        max_norm=0
        for i in range(len(first)):
            if max_norm<abs(first[i]-second[i]):
                max_norm=abs(first[i]-second[i])
        return max_norm

In [8]:
def get_distribution(mdp,trajectories):
    start_distribution=[0]*len(mdp.states)
    end_distribution=[0]*len(mdp.feature_matrix)
    for traj in trajectories:
        start_distribution[traj[0][0]]+=1
        end_distribution[mdp.state_action_index[traj[-1][0]][traj[-1][1]]]+=1
    return [x/len(trajectories) for x in start_distribution],[x/len(trajectories) for x in end_distribution]

In [9]:
def recover_policy(mdp,reward,start_distribution,end_distribution,discount_type='yes'):
    end_states=set()
    z_state=[0]*len(mdp.states)
    z_state_action=[0]*len(mdp.feature_matrix)
    z_end=1
    sign_po=True
    thre=0.00001
    last_diff=sys.maxsize
    e_discount=math.exp(mdp.discount-1)
    count_it=0
    last_p=[0]*len(mdp.feature_matrix)
    while sign_po:
        temp_z_s=[0]*len(mdp.states)
        for state in mdp.states:
            zs=0
            state_i=str(mdp.states.index(state))
            for action in mdp.state_action[state_i]:
                term=0
                index=mdp.state_action_index[int(state_i)][action]
                term=reward[index]
                for result in mdp.state_action_result[int(state_i)][action]:
                    if result=='end':
                        term+=0
                    else:
                        result_i=int(result)
                        if z_state[result]>0:
                            term+=math.log(z_state[result])*mdp.transition_matrix[int(state_i)][action][result_i]*mdp.discount
                z_state_action[index]=math.exp(term)
                
                zs+=z_state_action[index]
            temp_z_s[int(state_i)]=zs
        z_state=temp_z_s

        e_discount*=e_discount
        
        p_state_action=[]
        for state in range(len(mdp.states)):
            for action in mdp.state_action[str(state)]:
                if z_state[state]==0:
                    p_state_action.append(0)
                else:
                    p_state_action.append(z_state_action[mdp.state_action_index[state][action]]/z_state[state])
                    
        diff=judge_threshold(p_state_action,last_p)
        if diff<thre:
            sign_po=False
        last_p=p_state_action
        last_diff=diff
        count_it+=1


    policy={}
    for state_i in range(len(mdp.states)):
        p_state={}
        for action in mdp.state_action[str(state_i)]:
            p_state[action]=p_state_action[mdp.state_action_index[state_i][action]]
        policy[state_i]=p_state

    
    visits=svf_sovler(mdp,policy,start_distribution,discount_type=discount_type) 
    return policy,visits

In [10]:
"""
Randomize MDP example
"""
class mdp(object):
    """
    Randomize MDP.3
    """
    def get_feature_matrix(self):
        state_action_index=[]
        count=0
        feature_matrix=[]
        for state_s in self.states:
            state=self.states.index(state_s)
            state_action_index.append({})
            for action in self.state_action[str(state)]:
                state_action_index[state][action]=count
                count+=1
        for i in range(count):
            row=[0 for _ in range(count)]
            feature_matrix.append(row)
        for i in range(count):
            feature_matrix[i][i]=-1
        return state_action_index,feature_matrix
    

    def __init__(self,states,actions,state_action,discount,state_action_state,feature_matrix=None,mode='real feature'):

        if discount>1:
            raise ValueError('Unexpected Value')
        
        self.states=states
        self.actions=actions
        self.discount=discount
        
       
        self.state_action=state_action

        
        self.transition_matrix=state_action_state
        
        if mode=='real feature':

            self.feature_matrix=feature_matrix

            self.state_action_index=[]
            i=0
            for j in range(len(states)):
                action_index={}
                for ac in state_action[str(j)]:
                    action_index[ac]=i
                    i+=1
                self.state_action_index.append(action_index)
        elif mode=='synthetic feature':

            self.state_action_index,self.feature_matrix=self.get_feature_matrix()
        
        self.state_action_result=[]
        for i in range(len(states)):
            action_result={}
            for action in state_action[str(i)]:
                sum_p=0
                result=[]
                for j in range(len(states)):
                    prob=state_action_state[i][action][j]
                    sum_p+=prob
                    if prob>0:
                        result.append(j)
                if sum_p==0:
                    result=['end']
                action_result[action]=result
            self.state_action_result.append(action_result)

In [11]:
def generate_state(action):
    return 'grid_id:'+str(action['grid_id'])+',time_id:'+str(get_timeslot(action['time']))


def generate_state_new(action):
    try:
        if action['type']=='bus_off' or action['type']=='subway_off':
            return 'grid_id:'+str(action['grid_id'])+',time_id:'+str(get_timeslot(action['time']))+',off'
        else:
            return 'grid_id:'+str(action['grid_id'])+',time_id:'+str(get_timeslot(action['time']))+',on'
    except:
        print(action)
        raise(ValueError)
def generate_action(action,result_grid):
    if action['type']=='bus_off' or action['type']=='subway_off':
        return 'walk'
    else:
        return action['route']+' '+str(result_grid)
def generate_action_new(action,result_grid):
    if action['type']=='bus_off' or action['type']=='subway_off':
        return 'walk'+' '+str(result_grid)
    else:
        return action['route']+' '+str(result_grid)

In [12]:
def get_timeslot(time):
    formate='%Y-%m-%d %X'
    bill_time=datetime.strptime(time,formate)
    re=(bill_time.hour-6)*4+(bill_time.minute)//15
    if bill_time.minute%15!=0 or (bill_time.minute%15==0 and bill_time.second!=0):
        re=re+1
    return re

In [13]:
def get_mdp_input(mon,day):
    path_string='/home/guojun/Python_Output/mdp/2016'+mon+'//'+day+'//'
    states=json.load(open(path_string+'states.json'))
    actions=json.load(open(path_string+'actions.json'))
    state_action=json.load(open(path_string+'state_action.json'))
    state_action_state=json.load(open(path_string+'state_action_state.json'))
    sequences=json.load(open('/home/guojun/Python_Output/trajectories/agent_1//'+mon+day+'.json'))
        
    trajectories=[]
    for sequence in sequences:
        trajectory=[]
        looper=0
        for operation in sequence[:-1]:
            looper+=1
            state=generate_state(operation)
            action=generate_action(operation,sequence[looper]['grid_id'])
            trajectory.append([states.index(state),actions.index(action)])
        state=generate_state(sequence[-1])
        trajectory.append([states.index(state),actions.index('to end')])
        trajectories.append(trajectory)
    
    transition_matrix=[]
    for i in range(len(states)):
        transition_state={}
        for j in state_action[str(i)]:
            transition_action=[]
            for k in range(len(states)):
                if str(k) in state_action_state[str(i)][str(j)]:
                    transition_action.append(state_action_state[str(i)][str(j)][str(k)])
                else:
                    transition_action.append(0)
            transition_state[j]=transition_action
            
        transition_matrix.append(transition_state)
    state_action_index=[]
    i=0
    for j in range(len(states)):
        action_index={}
        for ac in state_action[str(j)]:
            action_index[ac]=i
            i+=1
        state_action_index.append(action_index)
        
  
    
    return [states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index]

In [14]:
def get_mdp_input_new(mon,day):
    path_string='/home/guojun/Python_Output/mdp/2016'+mon+'//'+day+'//new/'
    #path_string='/home/guojun/Python_Output/mdp/agent_new'+str(1)+'/2016'+mon+'//'+day+'/new/'
    states=json.load(open(path_string+'states.json'))
    actions=json.load(open(path_string+'actions.json'))
    state_action=json.load(open(path_string+'state_action.json'))
    state_action_state=json.load(open(path_string+'state_action_state.json'))
    #sequences=json.load(open('/home/guojun/Python_Output/trajectories/new/agent/'+mon+day+'_500.json'))
    sequences=json.load(open('/home/guojun/Python_Output/trajectories/'+mon+day+'.json'))
    true_sequences=[]
    for se in sequences:
        if len(se)>=2:
            true_sequences.append(se)

    sequences=true_sequences
        
    trajectories=[]
    for sequence in sequences:
        trajectory=[]
        looper=0
        for operation in sequence[:-1]:
            looper+=1
            state=generate_state_new(operation)
            action=generate_action_new(operation,sequence[looper]['grid_id'])
            trajectory.append([states.index(state),actions.index(action)])
        state=generate_state_new(sequence[-1])
        trajectory.append([states.index(state),actions.index('to end')])
        trajectories.append(trajectory)
    
    transition_matrix=[]
    for i in range(len(states)):
        transition_state={}
        for j in state_action[str(i)]:
            transition_action=[]
            for k in range(len(states)):
                if str(k) in state_action_state[str(i)][str(j)]:
                    transition_action.append(state_action_state[str(i)][str(j)][str(k)])
                else:
                    transition_action.append(0)
            transition_state[j]=transition_action
            
        transition_matrix.append(transition_state)
    state_action_index=[]
    i=0
    for j in range(len(states)):
        action_index={}
        for ac in state_action[str(j)]:
            action_index[ac]=i
            i+=1
        state_action_index.append(action_index)
        
  
    
    return [states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index]

In [15]:
def get_svf_from_trajectory(mdp,trajectories):
    max_len=0
    policies=[]
    sum_state=[0]*len(mdp.states)
    policy=[0]*len(mdp.feature_matrix)
    for traj in trajectories:
        for step in traj:
            feature_index=mdp.state_action_index[step[0]][step[1]]
            policy[feature_index]+=1
    for state in range(len(mdp.states)):
        for action in mdp.state_action[str(state)]:
            feature_index=mdp.state_action_index[state][action]
            policy[feature_index]/=len(trajectories)
    return policy

# Convergence Figures

In [35]:
mon='09'
day='05'
time0=datetime.now()
states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
rew_paper,diff_paper,reward_log,error_log,policy_log=irl(mdp=train_mdp,trajectories=trajectories,threshold=1e-14,learning_rate=1,model='paper',log='true')
print(datetime.now()-time0)

200
0.000123193128537
0.017330479040144856
400
1.34113238152e-05
0.006525335633047026
600
2.7292914366e-06
0.002893811477400314
800
7.10228259832e-07
0.0014306116759000766
1000
2.06255342166e-07
0.0007452664408327512
1200
6.34799026542e-08
0.00040133773314480353
1400
2.02602737542e-08
0.00022241296946682633
1600
6.63683665713e-09
0.00012557969841139085
1800
2.21940186085e-09
7.158534645326698e-05
2000
7.55260509302e-10
4.1284700772598045e-05
2200
2.60994973206e-10
2.3981540820206858e-05
2400
9.1442792998e-11
1.4015986750154697e-05
2600
3.24384105762e-11
8.230044002521108e-06
2800
1.16365444646e-11
4.8552781524582875e-06
3000
4.21633299188e-12
2.88177992091997e-06
0:01:30.327250


In [20]:
len(actions)

78

In [201]:
data=json.load(open('/home/guojun/Python_Output/convergence_log.json'))

In [202]:
data['irl_sp']=error_log[:3000]

In [203]:
json.dump(data,open('/opt/tomcat/webapps/visLineChart/003_IRL_GIS/data/IRL_Experiment_Convergence.json','w'))

In [297]:
thre=0.1
first=True
iter_log=[]
day='05'
whole_iter=0
while thre>9e-21:
    reward_total=[]
    visiting_total=[]
    feature_total=[]
    print(thre)
    states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
    train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
    if first:
        theta=np.random.random(size=(len(train_mdp.feature_matrix),))
    else:
        theta=rew_paper
    rew_paper,diff_paper,num_iter=irl(mdp=train_mdp,trajectories=trajectories,threshold=thre,learning_rate=1,model='paper',initial_input=theta,return_num_iter=True)
    whole_iter+=num_iter
    iter_this={'thre':thre,'num_iter':whole_iter}
    iter_log.append(iter_this)
    first=False
    thre=thre/10

0.1
2
0.01
2
0.001
2
0.0001
31
1e-05
66
1.0000000000000002e-06
118
1.0000000000000002e-07
188
1.0000000000000002e-08
200
2.44523626025e-06
0.002916118817698659
273
1.0000000000000003e-09
200
4.158299556e-07
0.0011874790512461893
351
1.0000000000000003e-10
200
5.24161161519e-08
0.000432450552083089
391
1.0000000000000003e-11
200
5.88585178946e-09
0.0001471261515003717
400
1.98212929094e-09
8.573435716154367e-05
413
1.0000000000000002e-12
200
6.32767330184e-10
4.8583929919173144e-05
400
2.196647966e-10
2.865942320077013e-05
428
1.0000000000000002e-13
200
6.67365747104e-11
1.5789436932935444e-05
400
2.3743379383e-11
9.39917417135836e-06
440
1.0000000000000002e-14
200
6.95799207172e-12
5.072710656911651e-06
400
2.5256506722e-12
3.0476805502949437e-06
449
1e-15
200
7.22795340045e-13
1.6248701985053214e-06
400
2.66573841913e-13
9.83550745318278e-07
457
1.0000000000000001e-16
200
7.46124219586e-14
5.174878387373458e-07
400
2.78562028838e-14
3.1484064845886427e-07
465
1e-17
200
7.59973329536e-

In [299]:
json.dump(iter_log,open('/opt/tomcat/webapps/visLineChart/003_IRL_GIS/data/IRL_Experiment_Convergence_Iter.json','w'))

In [131]:
import os

In [154]:
os.listdir('/home/guojun/Python_Output/mdp/agent_12//201609/')

['28', '06', '09', '05', '30', '26']

In [17]:
mon='09'
day='05'
states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)

In [19]:
features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))

In [33]:
feature_re=np.array(features).T

In [34]:
train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,feature_matrix=feature_re,discount=1,mode='real feature')

In [35]:
train_mdp.feature_matrix

array([[  200,   399,  6789, ...,     4,  1223,     0],
       [  200,  1321,  6743, ...,     2,     0,     0],
       [  200, 20091,  6717, ...,     2,     0,     0],
       ..., 
       [  200,   325,  3541, ...,     4,  1643,     0],
       [  200,  7281,  3514, ...,     2,     0,     0],
       [  200,   198,  1573, ...,     2,     0,     0]])

In [39]:
rew_paper,diff_paper,reward_log,error_log,policy_log=irl(mdp=train_mdp,trajectories=trajectories,threshold=1e-14,learning_rate=1,model='paper',log='true')

OverflowError: math range error

# Test error 

In [176]:
t1=datetime.now()
mon='09'
days=['05','06','09','26','28','30']
reward_total=[]
visiting_total=[]
feature_total=[]
for day in days:
    print(day)
    states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
    train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
    rew_paper,diff_paper=irl(mdp=train_mdp,trajectories=trajectories,threshold=1e-15,learning_rate=1,model='paper')
    rewards=list(rew_paper)
    reward_total.extend(rewards)
    svf=get_svf_from_trajectory(train_mdp,trajectories)
    visiting_total.extend(svf)
    
    state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/state_action_index.json'))
    #state_action_index=json.load(open('/home/guojun/Python_Output/mdp/agent_new'+str(1)+'/2016'+mon+'/'+day+'/new/state_action_index.json'))
    print(len(reward_total))
    print(' ')
t2=datetime.now()
print(t2-t1)

05
200
0.000116162949708
0.016983150238479048
400
1.46191309491e-05
0.006843164721304973
600
3.19947174322e-06
0.0032059416965797886
800
8.60159629909e-07
0.001631439028992536
1000
2.55721558023e-07
0.0008829688561273794
1200
8.06263032161e-08
0.0004992328357363694
1400
2.64195258961e-08
0.0002885458246304156
1600
8.8995504825e-09
0.00016829513351990905
1800
3.0622331623e-09
9.878189633123233e-05
2000
1.07197005299e-09
5.833898601062234e-05
2200
3.8070127516e-10
3.4588722004313576e-05
2400
1.36874266315e-10
2.058805430004694e-05
2600
4.97334714344e-11
1.2298150423831892e-05
2800
1.82361972747e-11
7.369922223489897e-06
3000
6.73957705206e-12
4.428931157298754e-06
3200
2.50765422202e-12
2.6686816913108252e-06
3400
9.38476260917e-13
1.6121651269802781e-06
3600
3.52969536979e-13
9.768027084208591e-07
258
 
06
200
9.4466519793e-05
0.010451004133285563
400
1.15788085108e-05
0.004350443301686334
600
2.30082034144e-06
0.002101210870045843
800
5.63036807118e-07
0.001072338507436444
1000
1.52859

In [177]:
mon='09'
features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+days[0]+'/new/features.json'))
#features=json.load(open('/home/guojun/Python_Output/mdp/agent_new'+str(1)+'//2016'+mon+'/'+days[0]+'/new/features.json'))
feature_total=[]
for i in range(len(features)):
    feature_total.append([])
for day in days:
    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
    #features=json.load(open('/home/guojun/Python_Output/mdp/agent_new'+str(1)+'/2016'+mon+'/'+day+'/new/features.json'))
    for i in range(len(features)):
        feature_total[i].extend(features[i])

In [178]:
station_index=['id','longitude','latitude','name']
subway_station_file='/home/menghai/167StationIDLocationsSorted.txt'
df_stations=pd.read_csv(subway_station_file,names=station_index)
stations={}
for i in range(len(df_stations)):
    station=df_stations.iloc[i]
    name =station['name']
    if station['name'] in ['前海湾站','后海站','大剧院站','购物公园站','深圳北站']:
        stations[name]={'latitude':station['latitude'],'longitude':station['longitude']}
        name=name[:-1]
        print(name)
    elif station['name'] in ['前海湾','后海','大剧院','购物公园','深圳北']:
        stations[name]={'latitude':station['latitude'],'longitude':station['longitude']}
        name=name+'站'
        print(name)
    stations[name]={'latitude':station['latitude'],'longitude':station['longitude']}

后海站
深圳北
大剧院站
购物公园
前海湾站


In [179]:
from sklearn import ensemble
from sklearn import utils
from sklearn import preprocessing
from sklearn import svm
from sklearn import linear_model

In [180]:
len(feature_total[0])

1613

In [181]:
feature_set=[0,1,2,3,4,5,6,7,8,9]

In [182]:
dfx=pd.DataFrame()
for i in feature_set:
    dfx[i]=feature_total[i]

In [183]:
mon='11'
day='02'
states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
test_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')

In [184]:
svf=get_svf_from_trajectory(test_mdp,trajectories)
df_test=pd.DataFrame()

state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/state_action_index.json'))
#state_action_index=json.load(open('/home/guojun/Python_Output/mdp/agent_new'+str(1)+'/2016'+mon+'/'+day+'/new/state_action_index.json'))

features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
#features=json.load(open('/home/guojun/Python_Output/mdp/agent_new'+str(1)+'/2016'+mon+'/'+day+'/new/features.json'))
valid=[]
for i in state_action_index:
    for j in state_action_index[i]:
        valid.append(state_action_index[i][j])
for i in feature_set:
    df_test[i]=features[i]

In [185]:
dfx

,0,1,2,3,4,5,6,7,8,9
0,200,399,6789,4,53,0,5,4,1223,0
1,200,1321,6743,4,53,0,10,2,0,0
2,200,20091,6717,4,53,0,4,2,0,0
3,200,989,7184,4,53,46,10,2,0,0
4,0,1342,6453,2,34,0,0,3,0,0
5,0,1104,6727,2,34,32,0,3,0,0
6,200,1520,5111,1,28,29,16,2,0,0
7,0,0,5111,1,29,0,0,1,5,0
8,200,214,2204,12,17,0,13,4,1460,0
9,100,1452,2645,12,17,0,5,2,0,0


In [186]:
it=300

In [163]:
#IRL LR
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,reward_total)
    model=linear_model.LinearRegression()
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    start,end=get_distribution(test_mdp,trajectories)
    po,vi=recover_policy(test_mdp,va,start,end,'no')
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(vi[k]-svf[k])
        truth+=svf[k]
print(total_error/it,total_error/it/truth)

2429.65706919 1174.79970165


In [164]:
for ri in ry:
    if abs(ri)>3:
        print(list(ry).index(ri),ri)

In [165]:
#IRL Lasso
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,reward_total)
    model=linear_model.Lasso()
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    va=0-va
    start,end=get_distribution(test_mdp,trajectories)
    po,vi=recover_policy(test_mdp,va,start,end,'no')
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(vi[k]-svf[k])
        truth+=svf[k]
print(total_error/it,total_error/it/truth)

1.19278853112 0.576742960256


In [166]:
model.coef_

array([  4.59012497e-04,  -3.50237997e-05,   2.37446268e-06,
         0.00000000e+00,  -0.00000000e+00,  -0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -0.00000000e+00,
        -0.00000000e+00])

In [167]:
#IRL RF
for min_leaf in range(1,80):
    total_error=0
    it=50
    whole_error=0
    t1=datetime.now()
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=min_leaf)
        re=model.fit(dfx,reward_total)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in range(len(vi)):
            whole_error+=abs(vi[k]-svf[k])
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth,whole_error/it/sum(svf))
    t2=datetime.now()
    print(t2-t1)

0.867919570124 0.41966072701 0.41966072701
0:00:06.733150
0.92159974839 0.445616430064 0.445616430064
0:00:05.945685
0.953070108097 0.460833132727 0.460833132727
0:00:05.582368
0.959990312569 0.464179223932 0.464179223932
0:00:05.324610
0.964141031551 0.46618619993 0.46618619993
0:00:05.148606
0.969584787419 0.468818391465 0.468818391465
0:00:05.013042
0.977120987918 0.472462332089 0.472462332089
0:00:04.919812
0.97870231148 0.473226941413 0.473226941413
0:00:04.823155
0.986079736993 0.476794110378 0.476794110378
0:00:04.738424
0.987871077443 0.477660268097 0.477660268097
0:00:04.670712
0.995905623699 0.481545171306 0.481545171306
0:00:04.620826
0.997858179817 0.482489280816 0.482489280816
0:00:04.573942
0.999757369574 0.483407586361 0.483407586361
0:00:04.514947
1.0021909593 0.4845842876 0.4845842876
0:00:04.479605
1.00579786355 0.486328315632 0.486328315632
0:00:04.439564
1.00697311515 0.486896579051 0.486896579051
0:00:04.406673
1.00844048221 0.487606087568 0.487606087568
0:00:04.36

In [149]:
#IRL SVM
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,reward_total)
    model=svm.SVR(kernel='rbf',degree=1)
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    va=0-va
    start,end=get_distribution(test_mdp,trajectories)
    po,vi=recover_policy(test_mdp,va,start,end,'no')
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(vi[k]-svf[k])/svf[k]
        truth+=svf[k]
print(total_error/it,total_error/it)

4.3381163793 4.3381163793


In [40]:
print(total_error/it,total_error/it/truth)

7.26654307395 48.6941909438


In [188]:
#ML LR
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,visiting_total)
    model=linear_model.LinearRegression()
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(va[k]-svf[k])
        truth+=svf[k]
print(total_error/it,total_error/it/truth)

2.02041666284 0.976921773373


In [40]:
#ML Lasso
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,visiting_total)
    model=linear_model.Lasso()
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(va[k]-svf[k])
        truth+=svf[k]
print(total_error/it,total_error/it/truth)

NameError: name 'it' is not defined

In [187]:
#ML RF
for min_leaf in range(1,80):
    total_error=0
    it=3
    whole_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,visiting_total)
        model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=min_leaf)
        re=model.fit(dfx,visiting_total)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in range(len(va)):
            whole_error+=abs(va[k]-svf[k])
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth,whole_error)

1.67636586282 0.810564643248 5.02909758845
1.69361878633 0.818906861434 5.08085635898
1.67736813369 0.811049266177 5.03210440108
1.67719060144 0.81096342491 5.03157180431
1.66090620877 0.803089515505 4.98271862631
1.64387372102 0.794853883499 4.93162116307
1.64755976909 0.796636179537 4.94267930727
1.66139952041 0.803328043969 4.98419856123
1.63615552241 0.791121942255 4.90846656724
1.64949091087 0.797569934679 4.9484727326
1.65924857586 0.802288008709 4.97774572757
1.67571382874 0.810249368531 5.02714148621
1.63545528913 0.790783362023 4.90636586738
1.65670915412 0.801060135057 4.97012746236
1.63112434784 0.788689244053 4.89337304353
1.65975999385 0.802535292046 4.97927998155
1.65093299776 0.798267219607 4.95279899328
1.6678560357 0.806449929903 5.00356810709
1.64124095411 0.793580875131 4.92372286234
1.66373853901 0.804459017715 4.99121561704
1.66016387141 0.802730576903 4.98049161422
1.66977572992 0.807378149869 5.00932718976
1.65999834739 0.802650541918 4.97999504217
1.65450921647 

In [197]:
#ML SVM
total_error=0
for i in range(it):
    dfy,ry=utils.resample(dfx,visiting_total)
    model=svm.SVR(kernel='linear')
    re=model.fit(dfy,ry)
    va=model.predict(df_test)
    truth=0
    predict=0
    for k in valid:
        total_error+=abs(va[k]-svf[k])
        truth+=svf[k]
print(total_error/it,total_error/it/truth)

0.898782192889 7.72243120995


# Error with grediant

In [209]:
mon='09'
days=['05','06','09','26','28','30']
thetas={}

thre=0.1
first=True
rf_error_log=[]
feature_set=[0,1,2,3,4,5,6,7,8,9]
while thre>9e-26:
    reward_total=[]
    visiting_total=[]
    feature_total=[]
    for day in days:
        print(day)
        states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
        train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
        if first:
            theta=np.random.random(size=(len(train_mdp.feature_matrix),))
        else:
            theta=thetas[day]
        rew_paper,diff_paper=irl(mdp=train_mdp,trajectories=trajectories,threshold=thre,learning_rate=1,model='paper',initial_input=theta)
        thetas[day]=rew_paper
        rewards=list(rew_paper)
        reward_total.extend(rewards)
        svf=get_svf_from_trajectory(train_mdp,trajectories)
        visiting_total.extend(svf)
        state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/state_action_index.json'))
    first=False
    
    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+days[0]+'/new/features.json'))
    feature_total=[]
    for i in range(len(features)):
        feature_total.append([])
    for day in days:
        features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
        for i in range(len(features)):
            feature_total[i].extend(features[i])
    
    dfx=pd.DataFrame()
    for i in feature_set:
        dfx[i]=feature_total[i]

    mon_t='11'
    day='02'
    states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon_t,day)
    test_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
    svf=get_svf_from_trajectory(test_mdp,trajectories)
    df_test=pd.DataFrame()
    state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon_t+'/'+day+'/new/state_action_index.json'))
    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon_t+'/'+day+'/new/features.json'))
    valid=[]
    for i in state_action_index:
        for j in state_action_index[i]:
            if actions[int(j)].split(' ')[0] in stations and actions[int(j)].split(' ')[1]=='1299':
                valid.append(state_action_index[i][j])
    for i in feature_set:
        df_test[i]=features[i]

    
    #IRL RF
    total_error=0
    it=50
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=17,max_features=None)
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error={'threshold':thre,'rf_absolute_error':total_error/it,'rf_relative_error':total_error/it/truth}
    
    
    #IRL LR
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=linear_model.LinearRegression()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['linear_absolute']=total_error/it
    rf_error['linear_relative']=total_error/it/truth
    
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=linear_model.Lasso()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    
    rf_error['lasso_absolute']=total_error/it
    rf_error['lasso_relative']=total_error/it/truth
    
    
    rf_error_log.append(rf_error)
    
    thre=thre/10
    
    

05
06
09
26
28
30
0.0359019962613 0.294209865466
0.0580950357014 0.476077500358
0.0367010831611 0.300758226944
05
06
09
26
28
30
0.0356762553673 0.292359962815
0.0580911476836 0.47604563881
0.0366022324682 0.299948164772
05
06
09
26
28
30
0.035786339582 0.29326208151
0.0585186954936 0.479549309824
0.0365357061932 0.299402994908
05
06
09
26
28
30
0.0356434662779 0.292091262615
0.0594533982996 0.487209017234
0.0362816734847 0.297321246349
05
06
09
26
28
30
0.0337529466045 0.276598822175
0.0599484506976 0.491265875198
0.0366943745107 0.30070325086
05
06
09
26
28
30
0.0318569739597 0.261061695696
0.0602846564105 0.49402101552
0.0373240710297 0.305863491166
05
06
09
26
200
1.66082537927e-05
0.007223113432603279
28
200
3.65729515379e-05
0.007458997301356675
30
0.0283070779851 0.231970989722
0.060368455304 0.494707731128
0.0378746934276 0.310375734452
05
200
2.78773516992e-06
0.002995526090701005
06
200
2.37074188219e-06
0.0020086143025130157
09
200
2.27484436043e-06
0.0019662964075580908
26


In [273]:
for r in rf_error_log:
    r['baseline_relative']=0.339601179018
    r['baseline_absolute']=0.041441031354 

In [211]:
json.dump(rf_error_log,open('/opt/tomcat/webapps/visLineChart/003_IRL_GIS/data/IRL_Experiment_Gradient_All.json','w'))

In [210]:
rf_error_log

[{'lasso_absolute': 0.036701083161123638,
  'lasso_relative': 0.30075822694375343,
  'linear_absolute': 0.058095035701409621,
  'linear_relative': 0.47607750035830482,
  'rf_absolute_error': 0.035901996261336507,
  'rf_relative_error': 0.29420986546627709,
  'threshold': 0.1},
 {'lasso_absolute': 0.036602232468228697,
  'lasso_relative': 0.29994816477210789,
  'linear_absolute': 0.05809114768360131,
  'linear_relative': 0.4760456388097718,
  'rf_absolute_error': 0.035676255367340988,
  'rf_relative_error': 0.29235996281548265,
  'threshold': 0.01},
 {'lasso_absolute': 0.036535706193214619,
  'lasso_relative': 0.29940299490803146,
  'linear_absolute': 0.058518695493607401,
  'linear_relative': 0.47954930982423727,
  'rf_absolute_error': 0.035786339582039778,
  'rf_relative_error': 0.29326208150996236,
  'threshold': 0.001},
 {'lasso_absolute': 0.036281673484693376,
  'lasso_relative': 0.29732124634859119,
  'linear_absolute': 0.059453398299577513,
  'linear_relative': 0.4872090172342001

# Experiment days of data

In [118]:
mon='09'
days=['05','06','09','26','28','30']

In [119]:
mon='09'
days=['05','06','09','26','28','30']

marker={}
reward_total=[]
visiting_total=[]
states,actions,state_action,state_action_state,trajectories,transition,state_action_index=get_mdp_input_new(mon,days[0])
features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+days[0]+'/new/features.json'))
feature_total=[]
for i in range(len(features)):
    feature_total.append([])
for day in days:
    print(day)
    states,actions,state_action,state_action_state,trajectories,transition,state_action_index=get_mdp_input_new(mon,day)
    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
    train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition,discount=1,mode='synthetic feature')
    rew_paper,diff_paper=irl(mdp=train_mdp,trajectories=trajectories,threshold=1e-15,learning_rate=1,model='paper')
    rewards=list(rew_paper)
    marker[day]={'start':len(reward_total),'end':len(reward_total)+len(rewards)}
    reward_total.extend(rewards)
    svf=get_svf_from_trajectory(train_mdp,trajectories)
    visiting_total.extend(svf)
    state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/state_action_index.json'))
    for i in range(len(features)):
        feature_total[i].extend(features[i])
    print(' ')

05
200
0.002056812902
0.034068651908653685
400
0.00198825096287
0.03449353675265029
600
0.0019793380683
0.03528182188202079


KeyboardInterrupt: 

In [214]:
mon='11'
day='02'
states,actions,state_action,state_action_state,trajectories,transition,state_action_index=get_mdp_input_new(mon,day)
features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
test_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition,discount=1,mode='synthetic feature')    

svf=get_svf_from_trajectory(test_mdp,trajectories)
df_test=pd.DataFrame()
state_action_index=json.load(open('/home/guojun/Python_Output/mdp//2016'+mon+'/'+day+'/new/state_action_index.json'))
features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
valid=[]
for i in state_action_index:
    for j in state_action_index[i]:
        if actions[int(j)].split(' ')[0] in stations and actions[int(j)].split(' ')[1]=='1299':
            valid.append(state_action_index[i][j])
for i in feature_set:
    df_test[i]=features[i]

In [215]:
lr=[]
la=[]
rf=[]
for ide in range(1,7):
    la_this=[]
    lr_this=[]
    rf_this=[]
    for ijn in range(50):
        vdays=list(np.random.choice(days,ide,False))
        test_features=[]
        test_reward=[]
        test_visiting=[]
        for i in range(len(feature_total)):
            test_features.append([])
        for day in vdays:
            for i in range(marker[day]['start'],marker[day]['end']):
                for j in range(len(feature_total)):
                    test_features[j].append(feature_total[j][i])
                test_reward.append(reward_total[i])
                test_visiting.append(visiting_total[i])
        dfx=pd.DataFrame()
        for i in feature_set:
            dfx[i]=test_features[i]




        total_error=0
        dfy,ry=utils.resample(dfx,test_reward)
        model=linear_model.LinearRegression()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
        lr_this.append(total_error/truth)


        total_error=0
        dfy,ry=utils.resample(dfx,test_reward)
        model=linear_model.Lasso()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
        la_this.append(total_error/truth)


        total_error=0

        it=2
        for i in range(it):
            dfy,ry=utils.resample(dfx,test_reward)
            model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=13)
            re=model.fit(dfy,ry)
            va=model.predict(df_test)
            va=0-va
            start,end=get_distribution(test_mdp,trajectories)
            po,vi=recover_policy(test_mdp,va,start,end,'no')
            truth=0
            predict=0
            for k in valid:
                total_error+=abs(vi[k]-svf[k])
                truth+=svf[k]
        rf_this.append(total_error/it/truth)
    lr.append(sum(lr_this)/len(lr_this))
    la.append(sum(la_this)/len(la_this))
    rf.append(sum(rf_this)/len(rf_this))
    print(sum(lr_this)/len(lr_this),sum(la_this)/len(la_this),(sum(rf_this)/len(rf_this)))

0.498390218519 0.321087908606 0.287703206947
0.49627106068 0.318549977015 0.277482767135
0.495297703678 0.315824752902 0.270706614217
0.488855423668 0.314313306901 0.266565427921
0.491213584845 0.313432271793 0.269674885288
0.487913464756 0.31258628523 0.264030796685


In [216]:
data={'irl_lr':lr,'lrl_lasso':la,'irl_rf':rf}

In [217]:
lr=[]
la=[]
rf=[]
for ide in range(1,7):
    la_this=[]
    lr_this=[]
    rf_this=[]
    for ijn in range(50):
        vdays=list(np.random.choice(days,ide,False))
        test_features=[]
        test_reward=[]
        test_visiting=[]
        for i in range(len(feature_total)):
            test_features.append([])
        for day in vdays:
            for i in range(marker[day]['start'],marker[day]['end']):
                for j in range(len(feature_total)):
                    test_features[j].append(feature_total[j][i])
                test_reward.append(reward_total[i])
                test_visiting.append(visiting_total[i])
        dfx=pd.DataFrame()
        for i in feature_set:
            dfx[i]=test_features[i]
           
        #ML LR
        total_error=0
        dfy,ry=utils.resample(dfx,test_visiting)
        model=linear_model.LinearRegression()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
        lr_this.append(total_error/truth) 
        

        #ML Lasso
        total_error=0
        dfy,ry=utils.resample(dfx,test_visiting)
        model=linear_model.Lasso()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
        la_this.append(total_error/truth)
        #ML RF
        total_error=0
        it=3
        for i in range(it):
            dfy,ry=utils.resample(dfx,test_visiting)
            model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=23,max_features=None)
            re=model.fit(dfy,ry)
            va=model.predict(df_test)
            truth=0
            predict=0
            for k in valid:
                total_error+=abs(va[k]-svf[k])
                truth+=svf[k]
        rf_this.append(total_error/it/truth)
    lr.append(sum(lr_this)/len(lr_this))
    la.append(sum(la_this)/len(la_this))
    rf.append(sum(rf_this)/len(rf_this))
    print(sum(lr_this)/len(lr_this),sum(la_this)/len(la_this),(sum(rf_this)/len(rf_this)))


0.460891377317 0.377605029424 0.322787070392
0.414795210089 0.368025032441 0.309070283126
0.44592577847 0.368751614264 0.320530049827
0.427377194374 0.362779813322 0.338381039977
0.412033329122 0.361771924266 0.341663902232
0.433763277804 0.366505529089 0.330652742143


In [218]:
data['ml_lr']=lr
data['ml_la']=la
data['ml_rf']=rf
json.dump(data,open('/home/guojun/Python_Output/days.json','w'))

# experiments with different days

In [26]:
mon='09'
days=['05','06','09','26','28','30']

In [117]:
reward_total=[]
visiting_total=[]
feature_total=[]
features=json.load(open('/home/guojun/Python_Output/mdp/2016/'+mon+'/'+days[0]+'/new/features.json'))
feature_total=[]
rf_error_log_days=[]
for i in range(len(features)):
    feature_total.append([])
for day in days:
    print(day)
    states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)
    train_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
    rew_paper,diff_paper=irl(mdp=train_mdp,trajectories=trajectories,threshold=1e-12,learning_rate=1,model='paper',)
    thetas[day]=rew_paper
    rewards=list(rew_paper)
    reward_total.extend(rewards)
    svf=get_svf_from_trajectory(train_mdp,trajectories)
    visiting_total.extend(svf)
    state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/state_action_index.json'))



    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon+'/'+day+'/new/features.json'))
    for i in range(len(features)):
        feature_total[i].extend(features[i])
    print(len(feature_total[9]))

dfx=pd.DataFrame()
for i in feature_set:
    dfx[i]=feature_total[i]

FileNotFoundError: [Errno 2] No such file or directory: '/home/guojun/Python_Output/mdp/201611/05/new/features.json'

In [212]:
mon_t='11'
days=['02','03','09','10','11']
rf_error_log_agents=[]
for day in days:
    states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon_t,day)
    test_mdp=mdp(states=states,actions=actions,state_action=state_action,state_action_state=transition_matrix,discount=1,mode='synthetic feature')
    svf=get_svf_from_trajectory(test_mdp,trajectories)
    df_test=pd.DataFrame()
    state_action_index=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon_t+'/'+day+'/new/state_action_index.json'))
    features=json.load(open('/home/guojun/Python_Output/mdp/2016'+mon_t+'/'+day+'/new/features.json'))
    valid=[]
    for i in state_action_index:
        for j in state_action_index[i]:
            if actions[int(j)].split(' ')[0] in stations and actions[int(j)].split(' ')[1]=='1299':
                valid.append(state_action_index[i][j])
    for i in feature_set:
        df_test[i]=features[i]


    #IRL RF
    total_error=0
    it=50
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=25,max_features=None)
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error={'threshold':thre,'rf_absolute_error':total_error/it,'rf_relative_error':total_error/it/truth}
    
    
    #IRL LR
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=linear_model.LinearRegression()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['linear_absolute']=total_error/it
    rf_error['linear_relative']=total_error/it/truth


    #IRL Lasso
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,reward_total)
        model=linear_model.Lasso()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        va=0-va
        start,end=get_distribution(test_mdp,trajectories)
        po,vi=recover_policy(test_mdp,va,start,end,'no')
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(vi[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['lasso_absolute']=total_error/it
    rf_error['lasso_relative']=total_error/it/truth
    
    
    #ML LR
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,visiting_total)
        model=linear_model.LinearRegression()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['ml_linear_absolute']=total_error/it
    rf_error['ml_linear_relative']=total_error/it/truth
 
    #ML Lasso
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,visiting_total)
        model=linear_model.Lasso()
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['ml_lasso_absolute']=total_error/it
    rf_error['ml_lasso_relative']=total_error/it/truth
    
    #ML RF
    total_error=0
    for i in range(it):
        dfy,ry=utils.resample(dfx,visiting_total)
        model=ensemble.RandomForestRegressor(n_estimators=20,min_samples_leaf=23,max_features=None)
        re=model.fit(dfy,ry)
        va=model.predict(df_test)
        truth=0
        predict=0
        for k in valid:
            total_error+=abs(va[k]-svf[k])
            truth+=svf[k]
    print(total_error/it,total_error/it/truth)
    rf_error['ml_rf_absolute']=total_error/it
    rf_error['ml_rf_relative']=total_error/it/truth
    

    
    
    print()
    rf_error_log_agents.append(rf_error)

0.0274753599727 0.225155222633
0.0602066544443 0.493381804602
0.0383378120106 0.314170901022
0.0509013208078 0.417126408178
0.0442793978907 0.362861039858
0.0410925975221 0.336745831642

0.028306744908 0.243214531907
0.0644450913627 0.553719008682
0.0355635014242 0.305565347764
0.0434526979237 0.373350154528
0.0416264611654 0.357658936066
0.033353463783 0.286576471714

0.0214919822207 0.190771527577
0.0586051391672 0.520202920698
0.0324730334442 0.28824378001
0.039027719197 0.346425822086
0.0408723418934 0.362799439279
0.0261406099645 0.232034627775

0.062519631717 0.335703239871
0.362584337811 1.94692024868
0.0949108791045 0.509630155192
0.103268021332 0.554504375413
0.0985595748288 0.529222064842
0.0854821547412 0.459002004806

0.0322255563931 0.215948268703
0.0722598084119 0.484223773611
0.0488543054128 0.32738000064
0.0716691470194 0.48026566336
0.0545135119849 0.365303189508
0.058953925048 0.395059060954



In [219]:
json.dump(rf_error_log_agents,open('/opt/tomcat/webapps/visLineChart/003_IRL_GIS/data/IRL_Experiment_Agents.json','w'))

In [130]:
len(json.load(open('/home/guojun/personal_project/001_POI_Scrapy/url_data/bus_stop/Shenzhen_Bus_Stations.json',encoding='latin')))

5253

In [27]:
mon='09'
day='06'
states,actions,state_action,state_action_state,trajectories,transition_matrix,state_action_index=get_mdp_input_new(mon,day)

In [28]:
sum_l=0
count_l=0
for traj in trajectories:
    if len(traj)>0:
        sum_l+=len(traj)
        count_l+=1

In [29]:
sum_l/1.0/count_l

2.1472081218274113

In [33]:
trajectories[4]

[[4, 9], [12, 3]]